In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import sklearn.model_selection as cv
import sklearn

In [ ]:
## Load dataset
data = pd.read_csv("datapreprocessed.csv", encoding='latin1')
data.head()

In [ ]:
def expand_categoric_column_with_binary_values(column_name):
    global data
    print(column_name)
    print(data[column_name].unique())
    data = pd.concat([
            data.drop(column_name, axis=1),
            pd.get_dummies(data[column_name], prefix=column_name, sparse=True)
        ], axis=1)

expand_categoric_column_with_binary_values('created')
expand_categoric_column_with_binary_values('tweet_created')
expand_categoric_column_with_binary_values('link_color')
expand_categoric_column_with_binary_values('sidebar_color')
expand_categoric_column_with_binary_values('tweet_location')
expand_categoric_column_with_binary_values('user_timezone')

data.head()

In [ ]:
def boolean_to_binary(column_name):
    global data
    data[column_name] = data[column_name].apply(lambda boolean: int(boolean))

boolean_to_binary('tweet_coord')

data.tweet_coord.sum()

In [ ]:
# All values must be numeric, label included
def genderToNumeric(gender) :
    if gender == 'male' :
        return 0
    elif gender == 'female' :
        return 1
    else :
        return 2


labels = data.gender.apply(lambda g: genderToNumeric(g))
datadt = data.drop('gender', axis=1)

print(data.shape)
datadt.head()

In [ ]:
data[data.gender.isnull()]

In [ ]:
# Cross-validation: split data into training and test sets (test 30% of data)
data_train, data_test, labels_train, labels_test = cv.train_test_split(datadt, labels, test_size=.3, random_state=1)

In [ ]:
data_train.info()
data.iloc[data_train.head().index]

In [ ]:
data_test.info()
data.iloc[data_test.head().index]

In [ ]:
from sklearn import tree
from sklearn.externals.six import StringIO  
import pydot
from IPython.display import Image 
from statsmodels.stats.proportion import proportion_confint

clf = tree.DecisionTreeClassifier(criterion='entropy')
pred = clf.fit(data_train, labels_train).predict(data_test)
print(sklearn.metrics.confusion_matrix(labels_test, pred))
print()
print("Accuracy on test set:", sklearn.metrics.accuracy_score(labels_test, pred))
print()
print(sklearn.metrics.classification_report(labels_test, pred))
epsilon = sklearn.metrics.accuracy_score(labels_test, pred)
print("Confidence interval: ",proportion_confint(count=epsilon*data_test.shape[0], nobs=data_test.shape[0], alpha=0.05, method='binom_test'))

#scores = sklearn.model_selection.cross_val_score(clf, Xn, y, cv=10)
#print("Accuracy on 10 fold cross-validation:", scores)
#print(np.mean(scores))

## Print tree
# dot_data = StringIO() 
# tree.export_graphviz(clf, out_file=dot_data,    
#                          filled=True, rounded=True,
#                          feature_names=list(datadt.columns.values),  
#                          special_characters=True)  
# graph = pydot.graph_from_dot_data(dot_data.getvalue())  
# Image(graph[0].create_png())  
